# Análisis de los vectores de banners

In [ ]:
# Módulos

import pandas as pd

import numpy as np
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.neighbors import NearestNeighbors
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
base_dir = Path().resolve().parents[1] # Nos situamos en la carpeta del proyecto
data_dir = base_dir / 'data'

ruta = Path(data_dir / f'info_imagenes.jsonl.gz')

if not ruta.exists():
    raise FileNotFoundError(f'No se encuentra la ruta: {ruta}')

df = pd.DataFrame(pd.read_json(ruta, lines=True, compression='infer'))
df.head()

In [ ]:
modelos = ['v_resnet', 'v_convnext', 'v_clip']

for mod in modelos:
    print(f"Procesando reducción de dimensionalidad para: {mod}...")
    
    matrix = np.vstack(df[mod].values)
    
    # PCA
    pca = PCA(n_components=3)
    coords_pca = pca.fit_transform(matrix)
    
    df[f'pca_{mod}_1'] = coords_pca[:, 0]
    df[f'pca_{mod}_2'] = coords_pca[:, 1]
    df[f'pca_{mod}_3'] = coords_pca[:, 2]
    
    # TSNE
    pca_pre = PCA(n_components=50)
    matrix_reduced = pca_pre.fit_transform(matrix)
    
    tsne = TSNE(n_components=3, perplexity=30, random_state=42, init='pca')
    coords_tsne = tsne.fit_transform(matrix_reduced)
    
    # Guardamos resultados con prefijo del modelo
    df[f'tsne_{mod}_1'] = coords_tsne[:, 0]
    df[f'tsne_{mod}_2'] = coords_tsne[:, 1]
    df[f'tsne_{mod}_3'] = coords_tsne[:, 2]

df.head()

In [ ]:
df_sample = df.sample(n=1000, random_state=42)

modelos_nombres = ['v_resnet', 'v_convnext', 'v_clip']
titulos_modelos = ['ResNet', 'ConvNeXt', 'CLIP']

fig = make_subplots(
    rows=2, cols=3,
    specs=[[{'type': 'scene'}]*3]*2,
    subplot_titles=(
        f'PCA - {titulos_modelos[0]}', f'PCA - {titulos_modelos[1]}', f'PCA - {titulos_modelos[2]}',
        f't-SNE - {titulos_modelos[0]}', f't-SNE - {titulos_modelos[1]}', f't-SNE - {titulos_modelos[2]}'
    ),
    horizontal_spacing=0.02,
    vertical_spacing=0.05
)

for i, mod in enumerate(modelos_nombres):
    # PCA
    fig.add_trace(
        go.Scatter3d(
            x=df_sample[f'pca_{mod}_1'],
            y=df_sample[f'pca_{mod}_2'],
            z=df_sample[f'pca_{mod}_3'],
            mode='markers',
            marker=dict(size=3, color=df_sample['brillo'], colorscale='Viridis', opacity=0.8),
            text=df_sample['id'],
            hoverinfo='text',
            name=f'PCA {mod}'
        ),
        row=1, col=i+1
    )
    
    # TSNE
    fig.add_trace(
        go.Scatter3d(
            x=df_sample[f'tsne_{mod}_1'],
            y=df_sample[f'tsne_{mod}_2'],
            z=df_sample[f'tsne_{mod}_3'],
            mode='markers',
            marker=dict(size=3, color=df_sample['brillo'], colorscale='Viridis', opacity=0.8),
            text=df_sample['id'],
            hoverinfo='text',
            name=f't-SNE {mod}'
        ),
        row=2, col=i+1
    )

fig.update_layout(
    title_text='Comparativa Global: Modelos (Columnas) vs Reducción (Filas)',
    height=1000, 
    width=1500,
    showlegend=False
)

fig.update_scenes(xaxis_showticklabels=False, yaxis_showticklabels=False, zaxis_showticklabels=False)

fig.show()

In [ ]:
matrix_resnet = np.vstack(df_sample['v_resnet'].values)
matrix_convnext = np.vstack(df_sample['v_convnext'].values)
matrix_clip = np.vstack(df_sample['v_clip'].values)

# 2. Función auxiliar actualizada para trabajar con las matrices directamente
def obtener_vecinos_modelo(matrix, query_idx_local):
    nn = NearestNeighbors(n_neighbors=6, metric='cosine')
    nn.fit(matrix)
    _, indices = nn.kneighbors(matrix[query_idx_local].reshape(1, -1))
    return df_sample.iloc[indices[0][1:]]['id'].tolist()

print("COMPARATIVA DE VECINOS POR MODELO (ESPACIO ORIGINAL)\n" + "="*60)

puntos_test = df_sample.sample(10, random_state=42)
for idx_pandas in puntos_test.index:
    idx_local = df_sample.index.get_loc(idx_pandas)
    id_objetivo = df_sample.iloc[idx_local]['id']
    
    # Calculamos vecinos para cada arquitectura
    vecinos_resnet = obtener_vecinos_modelo(matrix_resnet, idx_local)
    vecinos_convnext = obtener_vecinos_modelo(matrix_convnext, idx_local)
    vecinos_clip = obtener_vecinos_modelo(matrix_clip, idx_local)
    
    # Formateo de salida
    print(f"ID Objetivo: {id_objetivo}")
    print(f"  Vecinos (ResNet18 - Formas/Color):   {vecinos_resnet}")
    print(f"  Vecinos (ConvNeXt - Textura/Calidad): {vecinos_convnext}")
    print(f"  Vecinos (CLIP - Semántica/Estilo):    {vecinos_clip}")
    print("-" * 60)